In [1]:
!pip3 install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [3]:
import pandas as pd
import numpy as np
import datasets
from transformers import T5Tokenizer

In [4]:
raw_data = pd.read_csv("Sentence pairs in French-English - 2025-04-13.tsv", sep='\t', header=None,
                       names=[0,'input_to_translate',2,'label'])

In [5]:
def generate_prompt(x):
    language_mapping = {"en":"English", "es":"Spanish", "de":"German", "fr": "French", "it":"Italian", "pt":"Portuguese"}
    source_text = x["input_to_translate"]
    language = x["language_pair"].split('_')[1]
    input_text = f"Translate French to {language_mapping[language]}: {source_text}"
    return input_text

In [6]:
training_features = pd.concat([raw_data[['input_to_translate']],raw_data[['label']]],axis=1)
training_features["language_pair"] = "fr_en"
training_features["ID"] = range(len(training_features))
training_features["prompt"] = training_features.apply(generate_prompt, axis=1)

In [7]:
training_features.head(2)

,input_to_translate,label,language_pair,ID,prompt
0,Lorsqu'il a demandé qui avait cassé la fenêtre...,"When he asked who had broken the window, all t...",fr_en,0,Translate French to English: Lorsqu'il a deman...
1,Lorsqu'il a demandé qui avait cassé la fenêtre...,"Then, when he asked who had broken the window,...",fr_en,1,Translate French to English: Lorsqu'il a deman...


In [8]:
training_features.iloc[0]["prompt"]

"Translate French to English: Lorsqu'il a demandé qui avait cassé la fenêtre, tous les garçons ont pris un air innocent."

In [9]:
train_ds_raw = datasets.Dataset.from_pandas(training_features)


In [10]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [11]:
tokenized_source_training = train_ds_raw.map(
    lambda x: tokenizer(x["prompt"], truncation=True),
    batched=True, remove_columns=['ID', 'input_to_translate', 'label', 'language_pair', 'prompt'])

source_lengths_training = [len(x) for x in tokenized_source_training["input_ids"]]

Map:   0%|          | 0/417144 [00:00<?, ? examples/s]

In [12]:
tokenized_target_training = train_ds_raw.map(
    lambda x: tokenizer(x["label"], truncation=True),
    batched=True, remove_columns=['ID', 'input_to_translate', 'label', 'language_pair', 'prompt'])
target_lengths_training = [len(x) for x in tokenized_target_training["input_ids"]]

Map:   0%|          | 0/417144 [00:00<?, ? examples/s]

In [25]:
print(f"Min source length: {min(source_lengths_training)}")
print(f"Max source length: {max(source_lengths_training)}")
print(f"95% source length: {int(np.percentile(source_lengths_training, 95))}")
print(f"Min target length: {min(target_lengths_training)}")
print(f"Max target length: {max(target_lengths_training)}")
print(f"95% target length: {int(np.percentile(target_lengths_training, 95))}")

Min source length: 9
Max source length: 512
95% source length: 32
Min target length: 3
Max target length: 443
95% target length: 17


In [26]:
# reference: https://www.philschmid.de/fine-tune-flan-t5-deepspeed
def preprocess_function(sample, padding="max_length"):

    # tokenize inputs
    model_inputs = tokenizer(sample["prompt"], max_length=128, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["label"], max_length=128, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [27]:
tokenized_train_ds = train_ds_raw.map(
    preprocess_function, batched=True,
    remove_columns=['ID', 'input_to_translate', 'label', 'language_pair', 'prompt'])

Map:   0%|          | 0/417144 [00:00<?, ? examples/s]

In [17]:
tokenized_train_ds[0]

{'label': 'When he asked who had broken the window, all the boys put on an air of innocence.',
 'prompt': "Translate French to English: Lorsqu'il a demandé qui avait cassé la fenêtre, tous les garçons ont pris un air innocent.",
 'input_ids': [30355,
  15,
  2379,
  12,
  1566,
  10,
  15591,
  31,
  173,
  3,
  9,
  21088,
  285,
  3,
  3925,
  212,
  17230,
  50,
  25301,
  6,
  1739,
  110,
  27746,
  7,
  30,
  17,
  8060,
  73,
  799,
  16679,
  5,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1

In [28]:
ds_dict = tokenized_train_ds.train_test_split(test_size=0.2)
trainset = ds_dict["train"]
testset = ds_dict["test"]

In [19]:
trainset[0]

{'label': 'It took me an hour to learn the poem by heart.',
 'prompt': "Translate French to English: Cela m'a pris une heure d'apprendre le poème par cœur.",
 'input_ids': [30355,
  15,
  2379,
  12,
  1566,
  10,
  10074,
  3,
  51,
  31,
  9,
  8060,
  245,
  3,
  9419,
  3,
  26,
  31,
  17110,
  60,
  90,
  1977,
  5843,
  260,
  3,
  75,
  2,
  450,
  5,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,

In [20]:
testset[0]

{'label': 'Would you pass me the salt, please? "Here you are."',
 'prompt': "Translate French to English: «\xa0Pourrais-tu me passer le sel, s'il te plaît\xa0?\xa0» «\xa0Tiens.\xa0»",
 'input_ids': [30355,
  15,
  2379,
  12,
  1566,
  10,
  673,
  1474,
  8316,
  18,
  17,
  76,
  140,
  7460,
  90,
  142,
  40,
  6,
  3,
  7,
  31,
  173,
  3,
  17,
  15,
  3,
  31459,
  3,
  58,
  1168,
  673,
  332,
  8065,
  5,
  1168,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1

In [29]:
trainset.save_to_disk("tokenized_train_ds/train")
testset.save_to_disk("tokenized_train_ds/eval")

Saving the dataset (0/2 shards):   0%|          | 0/333715 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/83429 [00:00<?, ? examples/s]

In [30]:
!zip -r tatoeba_tokenized.zip tokenized_train_ds

updating: tokenized_train_ds/ (stored 0%)
updating: tokenized_train_ds/train/ (stored 0%)
updating: tokenized_train_ds/train/dataset_info.json (deflated 69%)
updating: tokenized_train_ds/train/state.json (deflated 47%)
updating: tokenized_train_ds/train/data-00000-of-00002.arrow (deflated 96%)
updating: tokenized_train_ds/train/data-00001-of-00002.arrow (deflated 96%)
updating: tokenized_train_ds/eval/ (stored 0%)
updating: tokenized_train_ds/eval/dataset_info.json (deflated 69%)
updating: tokenized_train_ds/eval/state.json (deflated 38%)
updating: tokenized_train_ds/eval/data-00000-of-00001.arrow (deflated 96%)


In [31]:
from google.colab import files
files.download('tatoeba_tokenized.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>